## Dimension Reduction Model

*** Note
1. Some of the torch version do not support "Weights=None". Instead, we change it to "pretrain = False" to support majority of the torch

In [1]:
# Allow auto-reloading of modules
%load_ext autoreload
%autoreload 2

# Standard libraries
import os
import logging

# Third-party libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# Fix for library conflicts
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Project-specific imports
from Dimension_Reduction_Model.predict import load_trained_model, predict_image_class
from Dimension_Reduction_Model.data_preparation import get_cifar10_datasets, get_dataloader


In [2]:
'''
Check CUDA availability
'''
print("CUDA available:", torch.cuda.is_available())
print("Current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CUDA available: True
Current device name: NVIDIA GeForce RTX 3070 Laptop GPU


In [3]:
'''
Load the dimension reduction model with the pre-trained PTH
'''
model = load_trained_model('./Dimension_Reduction_Model/output/resnet18_4cls_64dim_1tm_model.pth', 'resnet18', 64, 4, device)

C:\Users\micha\Anaconda3\envs\final_virtual\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\micha\Anaconda3\envs\final_virtual\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded successfully from ./Dimension_Reduction_Model/output/resnet18_4cls_64dim_1tm_model.pth


In [4]:
'''
Load the data
'''
train_dataset, test_dataset = get_cifar10_datasets(data_dir='./Dimension_Reduction_Model/data')
train_loader, test_loader = get_dataloader(train_dataset, test_dataset, batch_size=64, num_workers=8)

Filtering and remapping base training dataset...
Dataset filtered. New number of samples: 20000
Filtering and remapping base testing dataset...
Dataset filtered. New number of samples: 4000
Triplet Dataset created with 20000 samples (based on base dataset size).
Classes found in dataset: [0, 1, 2, 3]
Filtered CIFAR-10 Triplet train loader and standard test loader created with 8 workers.


In [5]:
'''
test data and its label, and show the mapping of the labeling of these 4 classes
'''
NEW_CLASS_MAPPING = {
    3: 0, # cat -> 0
    5: 1, # dog -> 1
    8: 2, # ship -> 2
    9: 3  # truck -> 3
}

N, H, W, C = test_dataset.data.shape
print(f"(In test data) Number of images: {N}, Height: {H}, Width: {W}, Channel: {C}")

(In test data) Number of images: 4000, Height: 32, Width: 32, Channel: 3


In [6]:
'''
Split data into ab_images/cd_images, and ab_labels/cd_labels
'''
ab_images, cd_images = [], []
ab_labels, cd_labels = [], []
# this flag the number of image off miss assign from the contrastive learning
ab_incorrect, cd_incorrect = 0,0
# denormalize the image back to its original vector
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

for image, label in test_dataset:
    pred_label, prob, _= predict_image_class(model, image, device, True)
    # denormalize the image, and put the image shape back to [3, 32, 32]
    denom_image = (image * torch.tensor(CIFAR10_STD ).view(1, 3, 1, 1) + torch.tensor(CIFAR10_MEAN ).view(1, 3, 1, 1)).squeeze(0).to(device)

    if pred_label in ["cat","dog"]:
        if label not in [0,1]:
            ab_incorrect += 1
        else:
            ab_images.append(denom_image)
            label = torch.tensor(label).long().to(device)
            ab_labels.append(label)
    elif pred_label in ["ship","truck"]:
        if label not in [2,3]:
            cd_incorrect += 1
        else:
            cd_images.append(denom_image)
            # all fix-match model only takes 0 or 1 as labels
            label = torch.tensor(label-2).long().to(device)
            cd_labels.append(label)

In [7]:
'''
Manager model output (based on the animal/vehicle category)
'''
print(f"Number of correct animal prediction: {len(ab_images)}")
print(f"Number of incorrect animal prediction: {ab_incorrect}")
print(f"Accuracy of correct animal prediction: {len(ab_images)/((len(ab_images) + cd_incorrect))}")
print("______________________________________________________________________________________________")
print(f"Number of correct vehicle prediction: {len(cd_images)}")
print(f"Number of incorrect vehicle prediction: {cd_incorrect}")
print(f"Accuracy of correct animal prediction: {len(cd_images)/((len(cd_images) + ab_incorrect))}")

Number of correct animal prediction: 1966
Number of incorrect animal prediction: 49
Accuracy of correct animal prediction: 0.983
______________________________________________________________________________________________
Number of correct vehicle prediction: 1951
Number of incorrect vehicle prediction: 34
Accuracy of correct animal prediction: 0.9755


In [8]:
'''
Double check the shape of each image (to make sure we fed the correct data to the Fix Match Model)
'''
denom_image.shape

torch.Size([3, 32, 32])

## Fix Match Model

*** Note
1. There are two fix match models we used for cat/dog and ship/truck classification

In [9]:


# Configure logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger()

# Constants (must match training)
CIFAR10_CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
SELECTED_CLASSES = [0, 1] # the code itself does not matter, just need to make sure the index 0 is cat/ship, 1 is dog/truck
#SELECTED_CLASSES = ['ship', 'truck']
NUM_CLASSES = len(SELECTED_CLASSES)
BATCH_SIZE = 64

class WideResNet(nn.Module):
    def __init__(self, num_classes=10, depth=28, widen_factor=2, dropout_rate=0.0):
        super().__init__()
        n_channels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
        assert (depth - 4) % 6 == 0
        n = (depth - 4) // 6
        self.conv1 = nn.Conv2d(3, n_channels[0], kernel_size=3, stride=1, padding=1, bias=True)
        self.block1 = self._make_block(n, n_channels[0], n_channels[1], 1, dropout_rate, True)
        self.block2 = self._make_block(n, n_channels[1], n_channels[2], 2, dropout_rate)
        self.block3 = self._make_block(n, n_channels[2], n_channels[3], 2, dropout_rate)
        self.bn1 = nn.BatchNorm2d(n_channels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(n_channels[3], num_classes)

    def _make_block(self, n, in_planes, out_planes, stride, dropout_rate=0.0, activate_before_residual=False):
        layers = []
        for i in range(int(n)):
            layers.append(BasicBlock(i == 0 and in_planes or out_planes,
                                     out_planes,
                                     i == 0 and stride or 1,
                                     dropout_rate,
                                     activate_before_residual))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        return self.fc(out)


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropout_rate=0.0, activate_before_residual=False):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=True)
        self.dropout_rate = dropout_rate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(
            in_planes, out_planes, kernel_size=1, stride=stride, padding=0, bias=True) or None
        self.activate_before_residual = activate_before_residual

    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual:
            out = self.relu1(self.bn1(x))
        else:
            out = self.bn1(x)
            out = self.relu1(out)
        out = self.conv1(out if self.equalInOut else x)
        out = self.bn2(out)
        out = self.relu2(out)
        if self.dropout_rate > 0:
            out = F.dropout(out, p=self.dropout_rate, training=self.training)
        out = self.conv2(out)
        shortcut = x if self.equalInOut else self.convShortcut(x)
        return torch.add(out, shortcut)

def evaluate(model, eval_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in eval_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    per_class_accuracy = []
    for i, class_name in enumerate(SELECTED_CLASSES):
        correct_class = sum((np.array(all_targets) == i) & (np.array(all_preds) == i))
        total_class = sum(np.array(all_targets) == i)
        accuracy = 100. * correct_class / total_class if total_class > 0 else 0.0
        per_class_accuracy.append(accuracy)

    return total_loss / total, 100. * correct / total, per_class_accuracy

class TensorDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.images)

# remvoe the toTensor part since the manager stage is passing the tensor version to the fix match model
def get_normalizer():
    return transforms.Compose([
        transforms.Normalize(**NORMALIZATION_VARIABLES)
    ])

In [10]:
# The normalize verctors for fix match model
NORMALIZATION_VARIABLES = {
    "mean": (0.4914, 0.4822, 0.4465),
    "std": (0.2470, 0.2435, 0.2616)
}

transform = get_normalizer()

ab_dataset = TensorDataset(ab_images, ab_labels, transform=transform)
ab_loader = DataLoader(ab_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

cd_dataset = TensorDataset(cd_images, cd_labels, transform=transform)
cd_loader = DataLoader(cd_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [11]:
ab_model = WideResNet(num_classes=2).to(device)
ab_model.load_state_dict(torch.load('Fix_Match_Model/best_model_ema.pth', map_location=device))

ab_loss, ab_acc, ab_per_class_acc = evaluate(ab_model, ab_loader, device)

cd_model = WideResNet(num_classes=2).to(device)
cd_model.load_state_dict(torch.load('Fix_Match_Model/best_model_ema_shipTruckFL.pth', map_location=device))

cd_loss, cd_acc, cd_per_class_acc = evaluate(cd_model, cd_loader, device)

# for the final accuracy calculation, we need to find the correct prediction/total iamges, which incluidng miss classified imaged from contrastive model
ab_size = len(ab_loader.dataset)
cd_size = len(cd_loader.dataset)
final_acc = (ab_acc * ab_size + cd_acc * cd_size)/(ab_size + cd_size + ab_incorrect + cd_incorrect)

print("Final accuracy: {:.2f}%".format(final_acc))

Final accuracy: 79.65%


In [12]:
ab_size = len(ab_loader.dataset)
cd_size = len(cd_loader.dataset)
final_acc = (ab_acc * ab_size + cd_acc * cd_size)/(ab_size + cd_size + ab_incorrect + cd_incorrect)
final_acc2 = (ab_acc * ab_size + cd_acc * cd_size)/(ab_size + cd_size)
print("Final accuracy (consider miss classification error from CL): {:.2f}%".format(final_acc))
print("Final accuracy2 (not consider classification error from CL): {:.2f}%".format(final_acc2))

Final accuracy (consider miss classification error from CL): 79.65%
Final accuracy2 (not consider classification error from CL): 81.34%


In [13]:
'''
The cat/dog specialist accuracy (consider CL error)
'''
(ab_acc * ab_size )/(ab_size + ab_incorrect)

69.4789081885856

In [14]:
'''
The ship/truck specialist accuracy (consider CL error)
'''
(cd_acc * cd_size)/(cd_size + cd_incorrect)

89.97481108312343